In [1]:
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
import torch
import numpy as np
import pandas as pd

import pandas as pd

df = pd.read_excel("D:\\Mestrado\\shoppingGPT\\shoppinggpt\\store_zara\\dataset.xlsx")
#df = pd.read_excel("D:\\MESTRADO\\1º Ano\\1º Semestre\\IACH\\shoppinggpt-iach\\shoppinggpt\\store_zara\\dataset.xlsx")




d:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODELO_FASHION = "openai/clip-vit-large-patch14" 
# Nota: Vamos chamá-lo de Fashion-CLIP, pois o processo é o mesmo que o fine-tuning

# 1. Carregar o Modelo e o Processador
try:
    model = CLIPModel.from_pretrained(MODELO_FASHION)
    processor = CLIPProcessor.from_pretrained(MODELO_FASHION)
    
    # Garantimos que o código usa a sua versão CPU instalada
    device = "cpu"
    model.to(device)
    print(f"Modelo {MODELO_FASHION} carregado com sucesso no dispositivo: {device}")
    
except Exception as e:
    print(f"ERRO CRÍTICO ao carregar o modelo: {e}")
    # Se este falhar, o problema é com a sua instalação do 'transformers' ou a conexão à internet.
    raise

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Modelo openai/clip-vit-large-patch14 carregado com sucesso no dispositivo: cpu


In [3]:
#NÃO PRECISO DE CORRER ISTO NOVAMENTE
#debug
import os
# Certifique-se de que todas as variáveis globais (model, processor, device, df) estão carregadas.
# ------------------------------------------------------------------------------------------------

def vetorizar_imagem_fashion(caminho_imagem):
    """Carrega, processa e devolve o vetor de embedding (768 dimensões) com debug."""
    
    # 🚨 DEBUG: Imprime o caminho da imagem que está a ser processada
    print(f"DEBUG: A processar imagem: {os.path.basename(caminho_imagem)}")
    
    if not caminho_imagem:
        # 🚨 DEBUG: Imprime se o caminho estiver vazio
        print("DEBUG: Caminho de imagem VAZIO encontrado. Skip.")
        return None
        
    try:
        image = Image.open(caminho_imagem).convert("RGB")
        
        # Pré-processamento
        inputs = processor(images=image, return_tensors="pt").to(device)
        
        # Geração do vetor
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        
        # Normalização e retorno como numpy array
        vetor = image_features.cpu().numpy().flatten()
        # 🚨 DEBUG: Confirma o sucesso da vetorização
        # print(f"DEBUG: Sucesso ao vetorizar {os.path.basename(caminho_imagem)}. Dimensão: {vetor.shape[0]}")
        return vetor
        
    except Exception as e:
        # 🚨 DEBUG: Imprime o erro se a imagem não puder ser carregada ou processada
        print(f"*** ERRO *** Falha Crítica ao processar {os.path.basename(caminho_imagem)}: {e}")
        return None 

# ------------------------------------------------------------------------------------------------

print("\n--- A vetorização com Fashion-CLIP está a começar... ---")

# Criamos uma função de callback para monitorizar o progresso
progresso_contador = 0
total_linhas = len(df)

def monitorizar_progresso(caminho_imagem):
    global progresso_contador
    
    # Chama a função de vetorização
    vetor = vetorizar_imagem_fashion(caminho_imagem)
    
    # Atualiza e imprime o progresso a cada 100 imagens
    progresso_contador += 1
    if progresso_contador % 100 == 0 or progresso_contador == 1:
        print(f"--- PROGRESSO: {progresso_contador}/{total_linhas} ({progresso_contador/total_linhas * 100:.1f}%) ---")
        
    return vetor

# 1. Aplicar a Vetorização ao DataFrame
print("\n--- A vetorização LVM está a começar... (Aguarde alguns minutos) ---")

# Nota: Em vez de .apply(vetorizar_imagem_fashion), usamos o monitor para controlo
df['vetor_clip'] = df['caminho_imagem_local'].apply(monitorizar_progresso)

print("\n--- Vetorização Concluída! ---")

# 2. Guardar o DataFrame com os vetores
FICHEIRO_VETORIZADO = 'zara_produtos_com_vetores_final.pkl'

# Usamos PKL (pickle) porque é o formato mais eficiente e rápido para guardar arrays NumPy (os vetores)
try:
    df.to_pickle(FICHEIRO_VETORIZADO)
    print(f"✅ DataFrame com {len(df)} vetores guardado como '{FICHEIRO_VETORIZADO}'")

    # 3. Verificação
    df_cheio = pd.read_pickle(FICHEIRO_VETORIZADO)
    vetor_amostra = df_cheio['vetor_clip'].dropna().iloc[0]
    
    print("\nVerificação dos Vetores:")
    print(f"Dimensão dos vetores: {vetor_amostra.shape[0]} (768 para CLIP Large)")
    print(f"Linhas com vetores preenchidos: {df_cheio['vetor_clip'].count()}")

except Exception as e:
    print(f"ERRO ao guardar ou verificar o ficheiro PKL: {e}")


--- A vetorização com Fashion-CLIP está a começar... ---

--- A vetorização LVM está a começar... (Aguarde alguns minutos) ---
DEBUG: A processar imagem: e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e.jpg
--- PROGRESSO: 1/3129 (0.0%) ---
DEBUG: A processar imagem: b42b0725-cfe2-4748-af3e-3abe590d83cd.jpg
DEBUG: A processar imagem: c27bdddf-2f9c-4693-976f-0d1272212c12.jpg
DEBUG: A processar imagem: 0cbc84c8-f9ff-4f02-940c-ff642b5af379.jpg


KeyboardInterrupt: 

vetorizar as descrições

In [ ]:
#NÃO PRECISO DE CORRER ISTO NOVAMENTE
# --- 2. Função de Vetorização de Texto com Debug ---

def vetorizar_texto(texto):
    """Vetoriza uma string de texto usando o codificador de texto do CLIP."""
    
    if pd.isna(texto) or not texto:
        # 🚨 DEBUG: Imprime se o texto estiver vazio
        print("DEBUG TEXTO: String de texto VAZIA encontrada. Skip.")
        return None
    
    # Processamento de texto exigido pelo CLIP
    inputs = processor(text=[texto], return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # Usa a função get_text_features do modelo CLIP
        text_features = model.get_text_features(**inputs)
        
    # Retorna o vetor como array numpy
    vetor = text_features.cpu().numpy().flatten()
    
    # 🚨 DEBUG: Imprime o sucesso da vetorização
    # print(f"DEBUG TEXTO: Sucesso ao vetorizar '{texto[:20]}...'")
    return vetor

# 3. Aplicação da Vetorização de Texto

print("\n--- A vetorização de texto LVM está a começar... ---")
# Nota: Esta fase é muito mais rápida do que a vetorização de imagens.
df['vetor_texto'] = df['description'].apply(vetorizar_texto)
print("Vetorização de texto concluída.")


--- A vetorização de texto LVM está a começar... ---
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada. Skip.
DEBUG TEXTO: String de texto VAZIA encontrada

Junção dos vetores

In [ ]:
#NÃO PRECISO DE CORRER ISTO NOVAMENTE
def criar_vetor_final_hibrido(row):
    """Cria o vetor de busca final como a média de imagem e texto."""
    vetor_img = row['vetor_clip']
    vetor_txt = row['vetor_texto']
    
    # 🚨 DEBUG: Verifica quais vetores existem para a média
    # print(f"DEBUG HÍBRIDO: Imagem: {'Sim' if vetor_img is not None else 'Não'}, Texto: {'Sim' if vetor_txt is not None else 'Não'}")

    # Prioridade 1: Média de Imagem e Texto (Melhor)
    if vetor_img is not None and vetor_txt is not None:
        return (vetor_img + vetor_txt) / 2
    # Prioridade 2: Apenas Imagem
    elif vetor_img is not None:
        return vetor_img 
    # Prioridade 3: Apenas Texto
    elif vetor_txt is not None:
        return vetor_txt
    else:
        return None

# Esta é a sua base de dados de vetores final:
df['vetor_final_busca'] = df.apply(criar_vetor_final_hibrido, axis=1)
print("\nCriação do vetor híbrido concluída.")


# --- 5. Gravar o Resultado Final ---

FICHEIRO_FINAL = 'zara_produtos_multimodais_final.pkl'
df.to_pickle(FICHEIRO_FINAL)
print(f"✅ Projeto guardado com sucesso com vetores híbridos em '{FICHEIRO_FINAL}'!")


Criação do vetor híbrido concluída.
✅ Projeto guardado com sucesso com vetores híbridos em 'zara_produtos_multimodais_final.pkl'!


Teste dos vetores

In [4]:
#PRECISO DE CORRER ISTO NOVAMENTE
# --- 2. CARREGAMENTO DA BASE DE DADOS VETORIAL ---
# (Feito em seguida para evitar o pico de memória inicial)
try:
    print("2. A carregar o DataFrame e os Vetores...")
    df_busca = pd.read_pickle('zara_produtos_multimodais_final.pkl')
    # O np.vstack exige a alocação de um grande bloco de memória contíguo
    VETORES_BUSCA = np.vstack(df_busca['vetor_final_busca'].values)
    print(f"Base de dados de {VETORES_BUSCA.shape[0]} vetores carregada.")
except Exception as e:
    print(f"ERRO CRÍTICO ao carregar os Vetores: {e}")
    sys.exit()

2. A carregar o DataFrame e os Vetores...
Base de dados de 3129 vetores carregada.


In [5]:
df_busca.head()

,brand,url,sku,name,description,price,currency,images,scraped_at,terms,section,error,image_downloads,ids_ficheiro_limpos,caminho_imagem_local,vetor_clip,vetor_texto,vetor_final_busca
0,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,272145190-250-2,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,19.99,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:05.654618,jackets,MAN,NaN,"['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3...","['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3...",images\imagens_zara\e8e4ae57-8b72-44ff-aa5d-84...,"[1.0900323, 1.1585298, 0.12433419, 0.89237976,...","[0.36500645, 0.81600964, -0.3475127, 0.4538082...","[0.7275194, 0.9872697, -0.11158925, 0.67309403..."
1,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,324052738-800-46,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,169.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:06.590930,jackets,MAN,NaN,"['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c...","['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c...",images\imagens_zara\b42b0725-cfe2-4748-af3e-3a...,"[1.2051622, 1.4055446, 0.3339182, 0.07867105, ...","[-0.27579236, 1.0658386, 0.05010604, -0.171342...","[0.4646849, 1.2356915, 0.19201213, -0.04633583..."
2,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,335342680-800-44,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,129.00,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:07.301419,jackets,MAN,NaN,"['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6...","['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6...",images\imagens_zara\c27bdddf-2f9c-4693-976f-0d...,"[1.2296686, 1.6341629, 0.030576326, 0.17466402...","[0.057577994, 1.2034019, -0.17215295, 0.119017...","[0.6436233, 1.4187825, -0.07078831, 0.14684062..."
3,Zara,https://www.zara.com/us/en/stretch-suit-jacket...,328303236-420-44,STRETCH SUIT JACKET,Slim fit jacket made of viscose blend fabric. ...,129.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:07.882922,jackets,MAN,NaN,"['0cbc84c8-f9ff-4f02-940c-ff642b5af379', 'dd47...","['0cbc84c8-f9ff-4f02-940c-ff642b5af379', 'dd47...",images\imagens_zara\0cbc84c8-f9ff-4f02-940c-ff...,"[1.3910327, 1.7873354, 0.2468657, -0.29693544,...","[0.2034662, 1.2178322, -0.24865599, 0.23475002...","[0.79724944, 1.5025837, -0.00089514256, -0.031..."
4,Zara,https://www.zara.com/us/en/double-faced-jacket...,312368260-800-2,DOUBLE FACED JACKET,Jacket made of faux leather faux shearling wit...,139.00,USD,['https://static.zara.net/photos///2024/V/0/2/...,2024-02-19T08:50:08.453847,jackets,MAN,NaN,"['1ccc9c87-49c9-4825-9d67-4c1f61e5db1f', '1349...","['1ccc9c87-49c9-4825-9d67-4c1f61e5db1f', '1349...",images\imagens_zara\1ccc9c87-49c9-4825-9d67-4c...,"[0.82264876, 1.138712, -0.27457723, 0.4562876,...","[0.28917527, 0.85195535, -0.3092939, 0.1881130...","[0.555912, 0.9953337, -0.29193556, 0.3222003, ..."


In [6]:
import os

df_busca["caminho_imagem_local"] = df_busca["caminho_imagem_local"].apply(
    lambda x: os.path.join("images\imagens_zara", os.path.basename(x))
)


In [7]:
df_busca.head()

,brand,url,sku,name,description,price,currency,images,scraped_at,terms,section,error,image_downloads,ids_ficheiro_limpos,caminho_imagem_local,vetor_clip,vetor_texto,vetor_final_busca
0,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,272145190-250-2,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,19.99,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:05.654618,jackets,MAN,NaN,"['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3...","['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3...",images\imagens_zara\e8e4ae57-8b72-44ff-aa5d-84...,"[1.0900323, 1.1585298, 0.12433419, 0.89237976,...","[0.36500645, 0.81600964, -0.3475127, 0.4538082...","[0.7275194, 0.9872697, -0.11158925, 0.67309403..."
1,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,324052738-800-46,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,169.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:06.590930,jackets,MAN,NaN,"['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c...","['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c...",images\imagens_zara\b42b0725-cfe2-4748-af3e-3a...,"[1.2051622, 1.4055446, 0.3339182, 0.07867105, ...","[-0.27579236, 1.0658386, 0.05010604, -0.171342...","[0.4646849, 1.2356915, 0.19201213, -0.04633583..."
2,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,335342680-800-44,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,129.00,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:07.301419,jackets,MAN,NaN,"['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6...","['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6...",images\imagens_zara\c27bdddf-2f9c-4693-976f-0d...,"[1.2296686, 1.6341629, 0.030576326, 0.17466402...","[0.057577994, 1.2034019, -0.17215295, 0.119017...","[0.6436233, 1.4187825, -0.07078831, 0.14684062..."
3,Zara,https://www.zara.com/us/en/stretch-suit-jacket...,328303236-420-44,STRETCH SUIT JACKET,Slim fit jacket made of viscose blend fabric. ...,129.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:07.882922,jackets,MAN,NaN,"['0cbc84c8-f9ff-4f02-940c-ff642b5af379', 'dd47...","['0cbc84c8-f9ff-4f02-940c-ff642b5af379', 'dd47...",images\imagens_zara\0cbc84c8-f9ff-4f02-940c-ff...,"[1.3910327, 1.7873354, 0.2468657, -0.29693544,...","[0.2034662, 1.2178322, -0.24865599, 0.23475002...","[0.79724944, 1.5025837, -0.00089514256, -0.031..."
4,Zara,https://www.zara.com/us/en/double-faced-jacket...,312368260-800-2,DOUBLE FACED JACKET,Jacket made of faux leather faux shearling wit...,139.00,USD,['https://static.zara.net/photos///2024/V/0/2/...,2024-02-19T08:50:08.453847,jackets,MAN,NaN,"['1ccc9c87-49c9-4825-9d67-4c1f61e5db1f', '1349...","['1ccc9c87-49c9-4825-9d67-4c1f61e5db1f', '1349...",images\imagens_zara\1ccc9c87-49c9-4825-9d67-4c...,"[0.82264876, 1.138712, -0.27457723, 0.4562876,...","[0.28917527, 0.85195535, -0.3092939, 0.1881130...","[0.555912, 0.9953337, -0.29193556, 0.3222003, ..."


In [8]:
FICHEIRO_FINAL = 'zara_produtos_multimodais_final.pkl'
df_busca.to_pickle(FICHEIRO_FINAL)
print(f"✅ Projeto guardado com sucesso com vetores híbridos em '{FICHEIRO_FINAL}'!")

✅ Projeto guardado com sucesso com vetores híbridos em 'zara_produtos_multimodais_final.pkl'!


In [9]:
#COMEÇAR A CORRER AQUI
# --- 2. FUNÇÕES ESSENCIAIS ---
from sklearn.metrics.pairwise import cosine_similarity
import os
def vetorizar_query(query_input, input_type='image'):
    """Vetoriza a query (imagem ou texto) com o modelo carregado."""
    
    with torch.no_grad():
        if input_type == 'image':
            image = Image.open(query_input).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to(device)
            features = model.get_image_features(**inputs)
        elif input_type == 'text':
            inputs = processor(text=[query_input], return_tensors="pt", padding=True, truncation=True).to(device)
            features = model.get_text_features(**inputs)
        else:
            return None
        
    return features.cpu().numpy().flatten().reshape(1, -1)


def executar_teste_recomendacao(query_input, input_type, n=5):
    """Executa a busca de similaridade, devolve ranking e abre a imagem."""
    
    vetor_query = vetorizar_query(query_input, input_type)
    if vetor_query is None:
        return pd.DataFrame()

    # Cálculo da Similaridade de Cosseno
    similaridades = cosine_similarity(vetor_query, VETORES_BUSCA).flatten()
    
    # Ranking e Resultados
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:n]
    
    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]
    
    return resultados







## TESTES

In [10]:

print("\n--- TESTE 1: BUSCA POR TEXTO (T2I) ---")
query_texto = "BLUE JEANS OVERSIZED FOR WOMEN"

# Alterado para n=5
resultados_texto = executar_teste_recomendacao(query_texto, input_type='text', n=5)

if not resultados_texto.empty:
    print(f"\n✅ ENCONTRADOS {len(resultados_texto)} RESULTADOS PARA: '{query_texto}'\n")

    # Itera sobre os 5 resultados para imprimir e abrir a imagem
    for i, produto in resultados_texto.iterrows():
        caminho_img = produto['caminho_imagem_local']
        
        print(f"[{i+1}] {produto['name']} ({produto['terms']})")
        print(f"    Preço: {produto['price']}, Similaridade: {produto['similaridade']:.4f}")
        
        # ABRIR IMAGEM: A forma de testar se a ligação funciona
        if os.path.exists(caminho_img):
            print(f"    A abrir imagem de teste: {os.path.basename(caminho_img)}")
            Image.open(caminho_img).show()
        else:
            print(f"    AVISO: Imagem não encontrada para visualização no caminho: {caminho_img}")
        
        print("-" * 50)
else:
    print("Nenhuma recomendação encontrada.")


## TESTE 2: I2I (Imagem para Imagem)
CAMINHO_IMAGEM_BUSCA = "D:\\Mestrado\\shoppingGPT\\shoppinggpt\\images\\imagens_zara\\20175e56-155c-40e4-aa75-45d9da713c1d.jpg"
'''
if os.path.exists(CAMINHO_IMAGEM_BUSCA):
    print("\n--- TESTE 2: BUSCA POR IMAGEM (I2I) ---")
    
    # O sistema procura produtos semelhantes ao ficheiro JPG de teste
    resultados_imagem = executar_teste_recomendacao(CAMINHO_IMAGEM_BUSCA, input_type='image', n=5)
    
    if not resultados_imagem.empty:
        produto_principal = resultados_imagem.iloc[0]
        caminho_img = produto_principal['caminho_imagem_local']

        print(f"Recomendação Principal (I2I): {produto_principal['name']}")
        print(f"Preço: {produto_principal['price']}, Similaridade: {produto_principal['similaridade']:.4f}")
        
        # ABRIR IMAGEM: A forma de testar se a ligação funciona
        if os.path.exists(caminho_img):
            print(f"A abrir imagem de teste I2I: {caminho_img}")
            Image.open(caminho_img).show() # Deverá abrir outra janela
        else:
            print(f"AVISO: Imagem não encontrada para visualização no caminho: {caminho_img}")

    print("-" * 50)
else:
    print("\nAVISO: Não foi possível realizar o Teste I2I. O CAMINHO_IMAGEM_BUSCA não é válido.")'''


--- TESTE 1: BUSCA POR TEXTO (T2I) ---

✅ ENCONTRADOS 5 RESULTADOS PARA: 'BLUE JEANS OVERSIZED FOR WOMEN'

[2342] TRF MID WAIST RIPPED CUFFED BOYFRIEND JEANS (jeans)
    Preço: 69.9, Similaridade: 0.5310
    A abrir imagem de teste: 20175e56-155c-40e4-aa75-45d9da713c1d.jpg
--------------------------------------------------
[380] STRAIGHT FIT JEANS (jeans)
    Preço: 69.9, Similaridade: 0.5275
    A abrir imagem de teste: 3c2f2b1a-b6df-4b7f-aaa7-111ed0605b43.jpg
--------------------------------------------------
[386] STRAIGHT LEG ACID WASH JEANS (jeans)
    Preço: 49.9, Similaridade: 0.5271
    A abrir imagem de teste: f42e5589-310a-4845-a147-bfbf30358e0c.jpg
--------------------------------------------------
[1913] ASYMMETRICAL RUCHED TOP (tops)
    Preço: 22.9, Similaridade: 0.5219
    A abrir imagem de teste: 273f5ffb-5854-492b-986e-e072ef199f86.jpg
--------------------------------------------------
[2339] Z1975 HIGH RISE WIDE LEG JEANS (jeans)
    Preço: 49.9, Similaridade: 0.5195

'\nif os.path.exists(CAMINHO_IMAGEM_BUSCA):\n    print("\n--- TESTE 2: BUSCA POR IMAGEM (I2I) ---")\n\n    # O sistema procura produtos semelhantes ao ficheiro JPG de teste\n    resultados_imagem = executar_teste_recomendacao(CAMINHO_IMAGEM_BUSCA, input_type=\'image\', n=5)\n\n    if not resultados_imagem.empty:\n        produto_principal = resultados_imagem.iloc[0]\n        caminho_img = produto_principal[\'caminho_imagem_local\']\n\n        print(f"Recomendação Principal (I2I): {produto_principal[\'name\']}")\n        print(f"Preço: {produto_principal[\'price\']}, Similaridade: {produto_principal[\'similaridade\']:.4f}")\n\n        # ABRIR IMAGEM: A forma de testar se a ligação funciona\n        if os.path.exists(caminho_img):\n            print(f"A abrir imagem de teste I2I: {caminho_img}")\n            Image.open(caminho_img).show() # Deverá abrir outra janela\n        else:\n            print(f"AVISO: Imagem não encontrada para visualização no caminho: {caminho_img}")\n\n    print

In [ ]:
## TESTE 2: I2I (Imagem para Imagem)

#CAMINHO_IMAGEM_BUSCA = "D:\\Mestrado\\shoppingGPT\\shoppinggpt\\images\\imagens_zara\\20175e56-155c-40e4-aa75-45d9da713c1d.jpg"
#CAMINHO_IMAGEM_BUSCA = "D:\\Mestrado\\shoppingGPT\\shoppinggpt\\ana\\foto3.jpeg"
CAMINHO_IMAGEM_BUSCA= "D:\\MESTRADO\\1º Ano\\1º Semestre\\IACH\\shoppinggpt-iach\\shoppinggpt\\ana\\foto3.jpeg"
if os.path.exists(CAMINHO_IMAGEM_BUSCA):
    print("\n--- TESTE 2: BUSCA POR IMAGEM (I2I) ---")
    
    # O sistema procura produtos semelhantes ao ficheiro JPG de teste
    resultados_imagem = executar_teste_recomendacao(CAMINHO_IMAGEM_BUSCA, input_type='image', n=5) # Pediu 5
    
    if not resultados_imagem.empty:
        
        print(f"✅ Encontradas {len(resultados_imagem)} recomendações:")
        
        # 🚨 CORREÇÃO APLICADA: Iterar sobre todas as linhas do DataFrame
        for rank, produto in resultados_imagem.iterrows():
            caminho_img = produto['caminho_imagem_local']

            print(f"\n[RANK {rank + 1}] Produto: {produto['name']}")
            print(f"    Preço: {produto['price']}, Similaridade: {produto['similaridade']:.4f}")
            
            # ABRIR IMAGEM: A forma de testar se a ligação funciona
            if os.path.exists(caminho_img):
                print(f"    A abrir imagem (Rank {rank + 1}): {os.path.basename(caminho_img)}")
                Image.open(caminho_img).show() # Abre uma nova janela para cada uma das 5 imagens
            else:
                print(f"    AVISO: Imagem não encontrada no caminho: {caminho_img}")
    
    print("-" * 50)
else:
    print("\nAVISO: Não foi possível realizar o Teste I2I. O CAMINHO_IMAGEM_BUSCA não é válido.")


AVISO: Não foi possível realizar o Teste I2I. O CAMINHO_IMAGEM_BUSCA não é válido.


Query Hibrida

In [16]:
def vetorizar_query_hibrida(query_texto, caminho_imagem):
    """
    Combina os vetores de texto e imagem numa única representação (vetor médio).
    """
    # 1. Vetorizar a Imagem (I)
    vetor_imagem = vetorizar_query(caminho_imagem, input_type='image')
    
    # 2. Vetorizar o Texto (T)
    vetor_texto = vetorizar_query(query_texto, input_type='text')
    
    if vetor_imagem is not None and vetor_texto is not None:
        # 3. Combinação: Cria o vetor híbrido como a média dos dois vetores (melhor para busca)
        vetor_hibrido = (vetor_imagem + vetor_texto) / 2
        return vetor_hibrido
    
    # Fallback: Se faltar um componente (embora a query híbrida exija ambos)
    elif vetor_imagem is not None:
        return vetor_imagem
    else:
        return None

In [17]:
# 🚨 CAMINHOS DE TESTE (Substitua pelos seus ficheiros locais)
IMAGEM_BUSCA_COMPLETA = "D:\\MESTRADO\\1º Ano\\1º Semestre\\IACH\\shoppinggpt-iach\\shoppinggpt\\ana\\foto4.jpg"
TEXTO_FOCO = "I WANT ONLY THE GREY SWEAT."


print("\n--- TESTE 3: BUSCA HÍBRIDA (Texto + Imagem) ---")

# 1. Gerar o Vetor Híbrido
vetor_final_hibrido = vetorizar_query_hibrida(TEXTO_FOCO, IMAGEM_BUSCA_COMPLETA)

if vetor_final_hibrido is not None:
    # 2. Executar a Busca de Similaridade (n=3)
    
    # Transforma o vetor híbrido no formato necessário para a comparação (1, 768)
    vetor_busca_formatado = vetor_final_hibrido.reshape(1, -1)
    
    # Cálculo da Similaridade de Cosseno (Reutilizando a lógica do seu motor)
    similaridades = cosine_similarity(vetor_busca_formatado, VETORES_BUSCA).flatten()
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:3]
    
    resultados_hibridos = df_busca.iloc[top_indices].copy()
    resultados_hibridos['similaridade'] = similaridades[top_indices]
    
    print(f"\n✅ RESULTADOS PARA QUERY HÍBRIDA ('{TEXTO_FOCO}'):\n")
    
    # 3. Mostrar e Abrir Imagens
    for rank, produto in resultados_hibridos.iterrows():
        caminho_img = produto['caminho_imagem_local']

        print(f"[RANK {rank + 1}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}, Categoria: {produto['terms']}")
        
        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank + 1}): {os.path.basename(caminho_img)}")
            Image.open(caminho_img).show()
        
        print("-" * 50)
else:
    print("AVISO: Falha ao gerar o vetor híbrido. Verifique os caminhos de imagem.")


--- TESTE 3: BUSCA HÍBRIDA (Texto + Imagem) ---

✅ RESULTADOS PARA QUERY HÍBRIDA ('I WANT ONLY THE GREY SWEAT.'):

[RANK 528] Produto: WASHED HOODIE SWEATSHIRT
    Similaridade: 0.8128, Categoria: hoodies
    A abrir imagem (Rank 528): 418c3632-4f8d-4ace-a508-71ec865ccfcb.jpg
--------------------------------------------------
[RANK 568] Produto: SOFT SWEATSHIRT
    Similaridade: 0.7991, Categoria: sweatshirts
    A abrir imagem (Rank 568): 7299972e-e317-48b5-adef-d7c59dda865d.jpg
--------------------------------------------------
[RANK 499] Produto: HOODIE SWEATSHIRT
    Similaridade: 0.7946, Categoria: hoodies
    A abrir imagem (Rank 499): 0fc651e2-71d6-4ea8-9431-3de4c66ccb97.jpg
--------------------------------------------------


In [18]:
from sklearn.metrics.pairwise import euclidean_distances

def executar_teste_recomendacao(query_input, input_type, n=3):
    """Executa a busca de similaridade, devolve ranking e abre a imagem."""
    
    vetor_query = vetorizar_query(query_input, input_type)
    if vetor_query is None:
        return pd.DataFrame()
    # Cálculo da Distância Euclidiana
    similaridades = euclidean_distances(vetor_query, VETORES_BUSCA).flatten()
    
    # Ranking e Resultados
    indices_ranking = np.argsort(similaridades)
    top_indices = indices_ranking[:n]
    
    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]
    
    return resultados
print("\n--- TESTE distância euclidiana: BUSCA POR TEXTO (T2I) ---")
query_texto = "black leather jacket"

# Alterado para n=5
resultados_texto = executar_teste_recomendacao(query_texto, input_type='text', n=5)

if not resultados_texto.empty:
    print(f"\n✅ ENCONTRADOS {len(resultados_texto)} RESULTADOS PARA: '{query_texto}'\n")

    # Itera sobre os 5 resultados para imprimir e abrir a imagem
    for i, produto in resultados_texto.iterrows():
        caminho_img = produto['caminho_imagem_local']
        
        print(f"[{i+1}] {produto['name']} ({produto['terms']})")
        print(f"    Preço: {produto['price']}, Similaridade: {produto['similaridade']:.4f}")
        
        # ABRIR IMAGEM: A forma de testar se a ligação funciona
        if os.path.exists(caminho_img):
            print(f"    A abrir imagem de teste: {os.path.basename(caminho_img)}")
            Image.open(caminho_img).show()
        else:
            print(f"    AVISO: Imagem não encontrada para visualização no caminho: {caminho_img}")
        
        print("-" * 50)
else:
    print("Nenhuma recomendação encontrada.")


--- TESTE distância euclidiana: BUSCA POR TEXTO (T2I) ---

✅ ENCONTRADOS 5 RESULTADOS PARA: 'black leather jacket'

[14] CROPPED LEATHER JACKET (jackets)
    Preço: 439.0, Similaridade: 12.1616
    A abrir imagem de teste: 8a71326a-0cb6-4c0a-b4e6-40a4b43f6eab.jpg
--------------------------------------------------
[1213] CROPPED LEATHER JACKET LIMITED EDITION (jackets)
    Preço: 349.0, Similaridade: 12.7618
    A abrir imagem de teste: 540a8f4c-05e3-45be-9bc5-a241fa62c4e0.jpg
--------------------------------------------------
[3022] DOUBLE FACED 100% LEATHER COAT (coats)
    Preço: 869.0, Similaridade: 12.8288
    A abrir imagem de teste: 68d97fc6-5aec-4bac-a092-c6efd62a3915.jpg
--------------------------------------------------
[34] FAUX LEATHER PUFFER JACKET (jackets)
    Preço: 109.0, Similaridade: 12.8593
    A abrir imagem de teste: a81258f9-d63e-41b8-9b10-922ad860e916.jpg
--------------------------------------------------
[46] FAUX LEATHER JACKET (jackets)
    Preço: 69.9, Simil

ANA

In [22]:
def vetorizar_query_imagem(caminho_imagem):
    # Só imagem
    return vetorizar_query(caminho_imagem, input_type='image')


IMAGEM_BUSCA = "D:\\MESTRADO\\1º Ano\\1º Semestre\\IACH\\shoppinggpt-iach\\shoppinggpt\\ana\\foto3.jpeg"

print("\n--- TESTE: BUSCA SÓ IMAGEM ---")

vetor_final = vetorizar_query_imagem(IMAGEM_BUSCA)

if vetor_final is not None:
    vetor_busca_formatado = vetor_final.reshape(1, -1)

    similaridades = cosine_similarity(vetor_busca_formatado, VETORES_BUSCA).flatten()
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:10]

    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]

    for rank, produto in resultados.iterrows():
        caminho_img = produto['caminho_imagem_local']

        print(f"[RANK {rank + 1}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}, Categoria: {produto['terms']}")

        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank + 1}): {os.path.basename(caminho_img)}")
            Image.open(caminho_img).show()
        print("-" * 50)
else:
    print("AVISO: Falha ao gerar vetor da imagem. Verifique o caminho.")



--- TESTE: BUSCA SÓ IMAGEM ---
[RANK 1341] Produto: nan
    Similaridade: 0.7508, Categoria: dresses
    A abrir imagem (Rank 1341): 45d02dc1-e35a-4b41-97ff-9ca9a9dbf214.jpg
--------------------------------------------------
[RANK 1324] Produto: COMBINATION LACE DRESS
    Similaridade: 0.6807, Categoria: dresses
    A abrir imagem (Rank 1324): f5125168-ce41-498d-8984-977297989ee9.jpg
--------------------------------------------------
[RANK 1389] Produto: nan
    Similaridade: 0.6793, Categoria: dresses
    A abrir imagem (Rank 1389): 64b643b0-6fdb-4483-81b3-02e5d526f7fa.jpg
--------------------------------------------------
[RANK 2476] Produto: nan
    Similaridade: 0.6728, Categoria: pants
    A abrir imagem (Rank 2476): 2b3921ec-d4b5-4f0b-8b90-3ac9126d9ba4.jpg
--------------------------------------------------
[RANK 1316] Produto: ASYMMETRICAL DRESS WITH RUCHING
    Similaridade: 0.6681, Categoria: dresses
    A abrir imagem (Rank 1316): 98427b90-5fed-4205-af1b-65e88ba5c7df.jpg
----

In [27]:
import cv2
import numpy as np

def cor_dominante_básica(caminho_imagem, k=3):
    """
    Devolve uma cor textual aproximada ('grey', 'black', 'white', 'red', etc.)
    com base na cor dominante da imagem (K-means em HSV).
    """
    img_bgr = cv2.imread(caminho_imagem)
    if img_bgr is None:
        return None

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # Achatar a imagem para (N, 3)
    pixels = img_hsv.reshape(-1, 3).astype(np.float32)

    # K-means para encontrar k cores dominantes
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Pega o centro mais frequente
    counts = np.bincount(labels.flatten())
    cor_hsv = centers[np.argmax(counts)]

    h, s, v = cor_hsv

    # Mapear HSV para nomes simples (regra heurística)
    if v < 40:
        return "black"
    if v > 200 and s < 40:
        return "white"
    if s < 40:
        return "grey"

    if h < 15 or h >= 165:
        return "red"
    if 15 <= h < 35:
        return "orange"
    if 35 <= h < 75:
        return "yellow/green"
    if 75 <= h < 130:
        return "blue"
    if 130 <= h < 165:
        return "purple"

    return "unknown"


In [28]:
import os
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

def vetorizar_query_imagem(caminho_imagem):
    return vetorizar_query(caminho_imagem, input_type='image')


def busca_por_imagem_auto_cor(caminho_imagem_query, df_busca, vetores_busca, top_k=5, usar_filtro_cor=True):
    """
    O utilizador só fornece a imagem.
    O sistema:
    - infere a cor dominante da imagem de query
    - opcionalmente filtra o dataset por essa cor
    - faz similaridade só com imagem.
    """
    # 1. Vetor da imagem
    vetor_query = vetorizar_query_imagem(caminho_imagem_query)
    if vetor_query is None:
        print("AVISO: Falha ao gerar vetor da imagem. Verifique o caminho.")
        return None

    vetor_query = vetor_query.reshape(1, -1)

    # 2. Inferir cor dominante da query
    cor_inferida = cor_dominante_básica(caminho_imagem_query)
    print(f"Cor dominante inferida na query: {cor_inferida}")

    # 3. Filtrar dataset por cor (se desejar e se houver coluna 'color')
    if usar_filtro_cor and 'color' in df_busca.columns and cor_inferida not in [None, "unknown"]:
        df_filtrado = df_busca[
            df_busca['color'].str.lower() == cor_inferida.lower()
        ].copy()
        if df_filtrado.empty:
            print(f"Nenhum produto encontrado com cor '{cor_inferida}'. A usar todo o dataset.")
            df_filtrado = df_busca.copy()
    else:
        df_filtrado = df_busca.copy()

    vetores_filtrados = vetores_busca[df_filtrado.index]

    # 4. Similaridade de cosseno
    similaridades = cosine_similarity(vetor_query, vetores_filtrados).flatten()
    indices_ranking_local = np.argsort(similaridades)[::-1]
    top_indices_local = indices_ranking_local[:top_k]
    indices_globais = df_filtrado.index[top_indices_local]

    resultados = df_busca.loc[indices_globais].copy()
    resultados['similaridade'] = similaridades[top_indices_local]

    # 5. Mostrar resultados
    print("\n--- RESULTADOS BUSCA POR IMAGEM (AUTO COR) ---\n")
    for rank, (idx, produto) in enumerate(resultados.iterrows(), start=1):
        caminho_img = produto['caminho_imagem_local']
        print(f"[RANK {rank}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}")
        print(f"    Cor (dataset): {produto.get('color', 'N/A')}")
        print(f"    Categoria: {produto.get('terms', 'N/A')}")

        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank}): {os.path.basename(caminho_img)}")
            try:
                Image.open(caminho_img).show()
            except Exception as e:
                print(f"    Erro ao abrir imagem: {e}")
        else:
            print("    Caminho de imagem não existe:", caminho_img)

        print("-" * 50)

    return resultados


In [29]:
IMAGEM_QUERY = "D:\\MESTRADO\\1º Ano\\1º Semestre\\IACH\\shoppinggpt-iach\\shoppinggpt\\ana\\foto3.jpeg"

resultados = busca_por_imagem_auto_cor(
    caminho_imagem_query=IMAGEM_QUERY,
    df_busca=df_busca,
    vetores_busca=VETORES_BUSCA,
    top_k=3,
    usar_filtro_cor=True
)


Cor dominante inferida na query: None

--- RESULTADOS BUSCA POR IMAGEM (AUTO COR) ---

[RANK 1] Produto: nan
    Similaridade: 0.7508
    Cor (dataset): N/A
    Categoria: dresses
    A abrir imagem (Rank 1): 45d02dc1-e35a-4b41-97ff-9ca9a9dbf214.jpg
--------------------------------------------------
[RANK 2] Produto: COMBINATION LACE DRESS
    Similaridade: 0.6807
    Cor (dataset): N/A
    Categoria: dresses
    A abrir imagem (Rank 2): f5125168-ce41-498d-8984-977297989ee9.jpg
--------------------------------------------------
[RANK 3] Produto: nan
    Similaridade: 0.6793
    Cor (dataset): N/A
    Categoria: dresses
    A abrir imagem (Rank 3): 64b643b0-6fdb-4483-81b3-02e5d526f7fa.jpg
--------------------------------------------------


In [5]:
import os
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1. Carregar modelo CLIP
# =========================
model_id = "openai/clip-vit-large-patch14"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

# =========================
# 2. Listas de descritores
# =========================
colors = ["black", "white", "red", "blue", "green", "yellow", "pink", "brown", "grey"]
tops = ["top", "t-shirt", "shirt", "blouse", "tank top", "sweater", "hoodie"]
bottoms = ["jeans", "shorts", "skirt", "trousers", "pants"]
dresses = ["dress"]
materials = ["cotton", "denim", "leather", "knit", "wool", "silk"]
top_lengths = ["cropped", "short", "regular", "long"]
bottom_lengths = ["short", "medium", "long", "above knee", "ankle length"]
dress_lengths = ["short", "medium", "long", "above knee", "ankle length"]
gender_descriptions = ["a woman", "a man", "a girl", "a boy"]

top_descriptions = [
    f"{color} {top}, {material}, {length}"
    for color in colors
    for top in tops
    for material in materials
    for length in top_lengths
]
bottom_descriptions = [
    f"{color} {bottom}, {material}, {length}"
    for color in colors
    for bottom in bottoms
    for material in materials
    for length in bottom_lengths
]
dress_descriptions = [
    f"{color} {dress}, {material}, {length}"
    for color in colors
    for dress in dresses
    for material in materials
    for length in dress_lengths
]

# =====================================
# 3. Funções auxiliares CLIP descrição
# =====================================
def process_descriptions(descriptions, image, processor, model, device, batch_size=64):
    matches = []
    for i in range(0, len(descriptions), batch_size):
        batch = descriptions[i:i+batch_size]
        inputs = processor(text=batch, images=image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
        for j, desc in enumerate(batch):
            matches.append((desc, probs[0][j].item()))
    matches.sort(key=lambda x: x[1], reverse=True)
    return matches

def contains_any(sentence, keywords):
    return any(word in sentence for word in keywords)

def gerar_descricao_roupa(image, question, processor, model, device, top_k=1):
    """
    Gera uma frase descritiva (TEXTO_FOCO) com base na imagem,
    combinando top/bottom/dress + género mais provável.
    """
    top_descr = None
    bottom_descr = None
    dress_descr = None

    found_any = False

    # Partes inferiores
    if contains_any(question, ["pant", "trouser", "bottom", "skirt", "short", "jean"]):
        found_any = True
        matches = process_descriptions(bottom_descriptions, image, processor, model, device)
        bottom_descr = ", ".join([m[0] for m in matches[:top_k]])

    # Partes superiores
    if contains_any(question, ["top", "shirt", "blouse", "hoodie", "sweater"]):
        found_any = True
        matches = process_descriptions(top_descriptions, image, processor, model, device)
        top_descr = ", ".join([m[0] for m in matches[:top_k]])

    # Vestidos
    if contains_any(question, ["dress"]):
        found_any = True
        matches = process_descriptions(dress_descriptions, image, processor, model, device)
        dress_descr = ", ".join([m[0] for m in matches[:top_k]])

    # Se o utilizador não especificar nada, assume top + bottom
    if not found_any:
        matches_bottom = process_descriptions(bottom_descriptions, image, processor, model, device)
        matches_top = process_descriptions(top_descriptions, image, processor, model, device)
        bottom_descr = matches_bottom[0][0]
        top_descr = matches_top[0][0]

    # Género
    gender_matches = process_descriptions(gender_descriptions, image, processor, model, device)
    gender_descr = gender_matches[0][0]

    # Montar frase final
    partes = []
    if top_descr:
        partes.append(f"top: {top_descr}")
    if bottom_descr:
        partes.append(f"bottom: {bottom_descr}")
    if dress_descr:
        partes.append(f"dress: {dress_descr}")

    descricao_roupa = "; ".join(partes)
    descricao_final = f"{gender_descr}, wearing {descricao_roupa}"

    return descricao_final  # TEXTO_FOCO

# =====================================
# 4. Vetorização e query híbrida
# =====================================
def vetorizar_query(texto_ou_imagem, input_type="text"):
    """
    Exemplo genérico; adapta para a tua implementação real.
    - input_type='text' => texto_ou_imagem é string
    - input_type='image' => texto_ou_imagem é caminho de imagem
    """
    if input_type == "text":
        inputs = processor(text=[texto_ou_imagem], return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            emb = model.get_text_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    elif input_type == "image":
        image = Image.open(texto_ou_imagem).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            emb = model.get_image_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    else:
        raise ValueError("input_type deve ser 'text' ou 'image'.")

def vetorizar_query_hibrida(query_texto, caminho_imagem):
    """
    Combina os vetores de texto e imagem numa única representação (vetor médio).
    """
    vetor_imagem = vetorizar_query(caminho_imagem, input_type='image')
    vetor_texto = vetorizar_query(query_texto, input_type='text')

    if vetor_imagem is not None and vetor_texto is not None:
        vetor_hibrido = (vetor_imagem + vetor_texto) / 2.0
        return vetor_hibrido
    elif vetor_imagem is not None:
        return vetor_imagem
    else:
        return None

# =====================================
# 5. Busca no catálogo (df_busca / VETORES_BUSCA)
# =====================================
def busca_hibrida_por_imagem(
    image_path,
    df_busca,
    vetores_busca,
    top_k=3,
    question="what are the clothes like?",
    top_k_descr=2
):
    """
    O utilizador só fornece a imagem.
    - Gera TEXTO_FOCO automaticamente com CLIP.
    - Cria vetor híbrido (texto + imagem).
    - Faz similaridade com VETORES_BUSCA e mostra top_k produtos.
    """
    # Verificar imagem
    if not os.path.exists(image_path):
        print("Imagem não encontrada:", image_path)
        return None

    image = Image.open(image_path).convert("RGB")

    # 1. Gerar TEXTO_FOCO com CLIP
    TEXTO_FOCO = gerar_descricao_roupa(
        image=image,
        question=question.lower(),
        processor=processor,
        model=model,
        device=device,
        top_k=top_k_descr
    )
    print("\nTEXTO_FOCO gerado automaticamente:")
    print(TEXTO_FOCO)

    # 2. Vetor híbrido
    vetor_hibrido = vetorizar_query_hibrida(TEXTO_FOCO, image_path)
    if vetor_hibrido is None:
        print("Falha ao gerar vetor híbrido.")
        return None

    vetor_busca_formatado = vetor_hibrido.reshape(1, -1)

    # 3. Similaridade com catálogo
    similaridades = cosine_similarity(vetor_busca_formatado, vetores_busca).flatten()
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:top_k]

    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]

    # 4. Mostrar resultados
    print("\n--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---\n")
    for rank, (idx, produto) in enumerate(resultados.iterrows(), start=1):
        caminho_img = produto['caminho_imagem_local']
        print(f"[RANK {rank}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}")
        print(f"    Categoria/terms: {produto.get('terms', 'N/A')}")

        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank}): {os.path.basename(caminho_img)}")
            try:
                Image.open(caminho_img).show()
            except Exception as e:
                print(f"    Erro ao abrir imagem: {e}")
        else:
            print("    Caminho de imagem não existe:", caminho_img)

        print("-" * 50)

    return resultados, TEXTO_FOCO

# =====================================
# 6. Exemplo de utilização interativa
# =====================================
if __name__ == "__main__":
    # Aqui assumes que já tens df_busca e VETORES_BUSCA carregados noutro módulo
    # e importas para este ficheiro, por exemplo:
    # from dados_catalogo import df_busca, VETORES_BUSCA

      # ajusta ao teu projeto

    while True:
        
        image_path = input("\nIndica o caminho da imagem para análise (ou 'sair'): ").strip()
        if image_path.lower() == "sair":
            break

        try:
            resultados, texto_foco = busca_hibrida_por_imagem(
            image_path=image_path,
            df_busca=df_busca,          # já definido antes no notebook
            vetores_busca=VETORES_BUSCA,  # já definido antes no notebook
            top_k=3,
            question="what are the clothes like?",
            top_k_descr=2)
        except Exception as e:
            print("Erro na busca híbrida:", e)





C:\Users\ritam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



TEXTO_FOCO gerado automaticamente:
a girl, wearing top: green t-shirt, cotton, cropped; bottom: grey pants, silk, ankle length

--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---

[RANK 1] Produto: CONTRASTING DOUBLE WAISTBAND PANTS
    Similaridade: 0.5689
    Categoria/terms: pants
    A abrir imagem (Rank 1): ef2c2c08-a780-4d2b-aac8-80d08d3b0d92.jpg
--------------------------------------------------
[RANK 2] Produto: POLYAMIDE T-SHIRT
    Similaridade: 0.5662
    Categoria/terms: tops
    A abrir imagem (Rank 2): 2fef5ef2-6587-44e2-bd4c-6a1d349e0283.jpg
--------------------------------------------------
[RANK 3] Produto: FULL LENGTH LOW WAIST Z1975 STRAIGHT LEG JEANS
    Similaridade: 0.5651
    Categoria/terms: jeans
    A abrir imagem (Rank 3): 8465ad29-c91e-4b0e-b6f2-b21c17689638.jpg
--------------------------------------------------

TEXTO_FOCO gerado automaticamente:
a man, wearing top: yellow shirt, wool, regular; bottom: grey pants, wool, ankle length

--- RESULTADOS B

In [6]:
import os
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1. Carregar modelo CLIP
# =========================
model_id = "openai/clip-vit-large-patch14"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

# =========================
# 2. Listas de descritores
# =========================
colors = ["black", "white", "red", "blue", "green", "yellow", "pink", "brown", "grey"]
tops = ["top", "t-shirt", "shirt", "blouse", "tank top", "sweater", "hoodie"]
bottoms = ["jeans", "shorts", "skirt", "trousers", "pants"]
dresses = ["dress"]
materials = ["cotton", "denim", "leather", "knit", "wool", "silk"]
top_lengths = ["cropped", "short", "regular", "long"]
bottom_lengths = ["short", "medium", "long", "above knee", "ankle length"]
dress_lengths = ["short", "medium", "long", "above knee", "ankle length"]
gender_descriptions = ["a woman", "a man", "a girl", "a boy"]

top_descriptions = [
    f"{color} {top}, {material}, {length}"
    for color in colors
    for top in tops
    for material in materials
    for length in top_lengths
]
bottom_descriptions = [
    f"{color} {bottom}, {material}, {length}"
    for color in colors
    for bottom in bottoms
    for material in materials
    for length in bottom_lengths
]
dress_descriptions = [
    f"{color} {dress}, {material}, {length}"
    for color in colors
    for dress in dresses
    for material in materials
    for length in dress_lengths
]

# =====================================
# 3. Funções auxiliares CLIP descrição
# =====================================
def process_descriptions(descriptions, image, processor, model, device, batch_size=64):
    """
    Recebe uma lista de descrições, calcula probs de matching com a imagem
    e devolve lista [(descricao, prob)] ordenada por prob desc.
    """
    matches = []
    for i in range(0, len(descriptions), batch_size):
        batch = descriptions[i:i+batch_size]
        inputs = processor(text=batch, images=image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
        for j, desc in enumerate(batch):
            matches.append((desc, probs[0][j].item()))
    matches.sort(key=lambda x: x[1], reverse=True)
    return matches

def gerar_descricao_roupa(image, user_focus, processor, model, device, top_k=1):
    """
    Gera TEXTO_FOCO com base na imagem e no que o utilizador quer:
    user_focus ∈ {"top", "bottom", "dress", "outfit"}.
    """
    top_descr = None
    bottom_descr = None
    dress_descr = None

    top_matches = []
    bottom_matches = []
    dress_matches = []

    # Partes inferiores
    if user_focus in ["bottom", "outfit"]:
        bottom_matches = process_descriptions(bottom_descriptions, image, processor, model, device)
        bottom_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in bottom_matches[:top_k]])

    # Partes superiores
    if user_focus in ["top", "outfit"]:
        top_matches = process_descriptions(top_descriptions, image, processor, model, device)
        top_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in top_matches[:top_k]])

    # Vestidos
    if user_focus == "dress":
        dress_matches = process_descriptions(dress_descriptions, image, processor, model, device)
        dress_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in dress_matches[:top_k]])

    # Género (sempre)
    gender_matches = process_descriptions(gender_descriptions, image, processor, model, device)
    gender_descr = f"{gender_matches[0][0]} ({gender_matches[0][1]:.3f})"

    # Montar frase final
    partes = []
    if top_descr:
        partes.append(f"top: {top_descr}")
    if bottom_descr:
        partes.append(f"bottom: {bottom_descr}")
    if dress_descr:
        partes.append(f"dress: {dress_descr}")

    descricao_roupa = "; ".join(partes) if partes else "clothes"
    descricao_final = f"{gender_descr}, wearing {descricao_roupa}"

    detalhes = {
        "top_matches": top_matches[:top_k],
        "bottom_matches": bottom_matches[:top_k],
        "dress_matches": dress_matches[:top_k],
        "gender_matches": gender_matches[:top_k],
    }

    return descricao_final, detalhes  # TEXTO_FOCO + detalhes

# =====================================
# 4. Vetorização e query híbrida
# =====================================
def vetorizar_query(texto_ou_imagem, input_type="text"):
    """
    input_type='text' => texto_ou_imagem é string
    input_type='image' => texto_ou_imagem é caminho de imagem
    """
    if input_type == "text":
        inputs = processor(text=[texto_ou_imagem], return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            emb = model.get_text_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    elif input_type == "image":
        image = Image.open(texto_ou_imagem).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            emb = model.get_image_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    else:
        raise ValueError("input_type deve ser 'text' ou 'image'.")

def vetorizar_query_hibrida(query_texto, caminho_imagem):
    """
    Combina os vetores de texto e imagem numa única representação (vetor médio).
    """
    vetor_imagem = vetorizar_query(caminho_imagem, input_type='image')
    vetor_texto = vetorizar_query(query_texto, input_type='text')

    if vetor_imagem is not None and vetor_texto is not None:
        vetor_hibrido = (vetor_imagem + vetor_texto) / 2.0
        return vetor_hibrido
    elif vetor_imagem is not None:
        return vetor_imagem
    else:
        return None

# =====================================
# 5. Busca no catálogo (df_busca / VETORES_BUSCA)
# =====================================
def busca_hibrida_por_imagem(
    image_path,
    df_busca,
    vetores_busca,
    top_k=3,
    user_focus="outfit",   # "top", "bottom", "dress", "outfit"
    top_k_descr=2
):
    """
    O utilizador só fornece a imagem e o foco (top/bottom/dress/outfit).
    - Gera TEXTO_FOCO automaticamente com CLIP.
    - Cria vetor híbrido (texto + imagem).
    - Faz similaridade com VETORES_BUSCA e mostra top_k produtos.
    """
    if not os.path.exists(image_path):
        print("Imagem não encontrada:", image_path)
        return None

    image = Image.open(image_path).convert("RGB")

    # 1. Gerar TEXTO_FOCO com CLIP
    TEXTO_FOCO, detalhes = gerar_descricao_roupa(
        image=image,
        user_focus=user_focus,
        processor=processor,
        model=model,
        device=device,
        top_k=top_k_descr
    )

    print("\nTEXTO_FOCO gerado automaticamente:")
    print(TEXTO_FOCO)

    print("\nTop descritores usados (texto, prob):")
    print("Top:", detalhes["top_matches"])
    print("Bottom:", detalhes["bottom_matches"])
    print("Dress:", detalhes["dress_matches"])
    print("Gender:", detalhes["gender_matches"])

    # 2. Vetor híbrido
    vetor_hibrido = vetorizar_query_hibrida(TEXTO_FOCO, image_path)
    if vetor_hibrido is None:
        print("Falha ao gerar vetor híbrido.")
        return None

    vetor_busca_formatado = vetor_hibrido.reshape(1, -1)

    # 3. Similaridade com catálogo
    similaridades = cosine_similarity(vetor_busca_formatado, vetores_busca).flatten()
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:top_k]

    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]

    print("\n--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---\n")
    for rank, (idx, produto) in enumerate(resultados.iterrows(), start=1):
        caminho_img = produto['caminho_imagem_local']
        print(f"[RANK {rank}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}")
        print(f"    Categoria/terms: {produto.get('terms', 'N/A')}")

        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank}): {os.path.basename(caminho_img)}")
            try:
                Image.open(caminho_img).show()
            except Exception as e:
                print(f"    Erro ao abrir imagem: {e}")
        else:
            print("    Caminho de imagem não existe:", caminho_img)

        print("-" * 50)

    return resultados, TEXTO_FOCO, detalhes

# =====================================
# 6. Loop interativo (imagem + escolha do utilizador)
# =====================================
if __name__ == "__main__":
    # Aqui assumes que df_busca e VETORES_BUSCA já estão definidos no mesmo ficheiro.
    # Se estiveres num notebook, ignora este bloco e chama busca_hibrida_por_imagem numa célula.

    while True:
        image_path = input("\nIndica o caminho da imagem (ou 'sair'): ").strip()
        if image_path.lower() == "sair":
            break

        if not os.path.exists(image_path):
            print("Imagem não encontrada, tenta de novo.")
            continue

        print("\nO que queres procurar?")
        print("1 - Parte de cima (top)")
        print("2 - Parte de baixo (bottom)")
        print("3 - Vestido (dress)")
        print("4 - Conjunto/outfit completo")
        opcao = input("Escolhe 1/2/3/4: ").strip()

        if opcao == "1":
            user_focus = "top"
        elif opcao == "2":
            user_focus = "bottom"
        elif opcao == "3":
            user_focus = "dress"
        elif opcao == "4":
            user_focus = "outfit"
        else:
            print("Opção inválida, a assumir 'outfit'.")
            user_focus = "outfit"

        try:
            resultados, texto_foco, detalhes = busca_hibrida_por_imagem(
                image_path=image_path,
                df_busca=df_busca,
                vetores_busca=VETORES_BUSCA,
                top_k=3,
                user_focus=user_focus,
                top_k_descr=2
            )
        except Exception as e:
            print("Erro na busca híbrida:", e)


C:\Users\ritam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



O que queres procurar?
1 - Parte de cima (top)
2 - Parte de baixo (bottom)
3 - Vestido (dress)
4 - Conjunto/outfit completo

TEXTO_FOCO gerado automaticamente:
a man (0.571), wearing top: yellow shirt, wool, regular (0.619), white hoodie, cotton, regular (0.532)

Top descritores usados (texto, prob):
Top: [('yellow shirt, wool, regular', 0.6190184950828552), ('white hoodie, cotton, regular', 0.5324385762214661)]
Bottom: []
Dress: []
Gender: [('a man', 0.5713792443275452), ('a boy', 0.4226285517215729)]

--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---

[RANK 1] Produto: HOODIE SWEATSHIRT
    Similaridade: 0.7609
    Categoria/terms: hoodies
    A abrir imagem (Rank 1): 0fc651e2-71d6-4ea8-9431-3de4c66ccb97.jpg
--------------------------------------------------
[RANK 2] Produto: WASHED HOODIE SWEATSHIRT
    Similaridade: 0.7374
    Categoria/terms: hoodies
    A abrir imagem (Rank 2): 736a7c27-1ad9-4610-bffc-4291e9a322b4.jpg
--------------------------------------------------
[RA

In [11]:
import os
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1. Carregar modelo CLIP
# =========================
model_id = "openai/clip-vit-large-patch14"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

# =========================
# 2. Listas de descritores
# =========================
colors = ["black", "white", "red", "blue", "green", "yellow", "pink", "brown", "grey"]
tops = ["top", "t-shirt", "shirt", "blouse", "tank top", "sweater", "hoodie"]
bottoms = ["jeans", "shorts", "skirt", "trousers", "pants"]
dresses = ["dress"]
materials = ["cotton", "denim", "leather", "knit", "wool", "silk"]
top_lengths = ["cropped", "short", "regular", "long"]
bottom_lengths = ["short", "medium", "long", "above knee", "ankle length"]
dress_lengths = ["short", "medium", "long", "above knee", "ankle length"]
gender_descriptions = ["a woman", "a man", "a girl", "a boy"]

top_descriptions = [
    f"{color} {top}, {material}, {length}"
    for color in colors
    for top in tops
    for material in materials
    for length in top_lengths
]
bottom_descriptions = [
    f"{color} {bottom}, {material}, {length}"
    for color in colors
    for bottom in bottoms
    for material in materials
    for length in bottom_lengths
]
dress_descriptions = [
    f"{color} {dress}, {material}, {length}"
    for color in colors
    for dress in dresses
    for material in materials
    for length in dress_lengths
]

# =====================================
# 3. Funções auxiliares CLIP descrição
# =====================================
def process_descriptions(descriptions, image, processor, model, device, batch_size=64):
    """
    Recebe uma lista de descrições, calcula probs de matching com a imagem
    e devolve lista [(descricao, prob)] ordenada por prob desc.
    """
    matches = []
    for i in range(0, len(descriptions), batch_size):
        batch = descriptions[i:i+batch_size]
        inputs = processor(text=batch, images=image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
        for j, desc in enumerate(batch):
            matches.append((desc, probs[0][j].item()))
    matches.sort(key=lambda x: x[1], reverse=True)
    return matches

def infer_user_focus(user_text: str) -> str:
    """
    Converte texto livre do utilizador (ex: 'sweat', 't-shirt', 'pants')
    num dos focos: 'top', 'bottom', 'dress', 'outfit'.
    """
    text = user_text.lower()

    keywords_top = [
        "top", "t-shirt", "tshirt", "shirt", "blouse", "tank", "tank top",
        "sweater", "hoodie", "jumper", "sweat", "sweatshirt", "pullover"
    ]
    keywords_bottom = [
        "jeans", "shorts", "skirt", "trousers", "pants", "bottom"
    ]
    keywords_dress = [
        "dress", "vestido"
    ]
    keywords_outfit = [
        "outfit", "conjunto", "look"
    ]

    if any(k in text for k in keywords_top):
        return "top"
    if any(k in text for k in keywords_bottom):
        return "bottom"
    if any(k in text for k in keywords_dress):
        return "dress"
    if any(k in text for k in keywords_outfit):
        return "outfit"

    # fallback: se não reconhece, assume outfit completo
    return "outfit"

def gerar_descricao_roupa(image, user_focus, processor, model, device, top_k=1):
    """
    Gera TEXTO_FOCO com base na imagem e no que o utilizador quer:
    user_focus ∈ {"top", "bottom", "dress", "outfit"}.
    """
    top_descr = None
    bottom_descr = None
    dress_descr = None

    top_matches = []
    bottom_matches = []
    dress_matches = []

    # Partes inferiores
    if user_focus in ["bottom", "outfit"]:
        bottom_matches = process_descriptions(bottom_descriptions, image, processor, model, device)
        bottom_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in bottom_matches[:top_k]])

    # Partes superiores
    if user_focus in ["top", "outfit"]:
        top_matches = process_descriptions(top_descriptions, image, processor, model, device)
        top_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in top_matches[:top_k]])

    # Vestidos
    if user_focus == "dress":
        dress_matches = process_descriptions(dress_descriptions, image, processor, model, device)
        dress_descr = ", ".join([f"{m[0]} ({m[1]:.3f})" for m in dress_matches[:top_k]])

    # Género (sempre)
    gender_matches = process_descriptions(gender_descriptions, image, processor, model, device)
    gender_descr = f"{gender_matches[0][0]} ({gender_matches[0][1]:.3f})"

    # Montar frase final
    partes = []
    if top_descr:
        partes.append(f"top: {top_descr}")
    if bottom_descr:
        partes.append(f"bottom: {bottom_descr}")
    if dress_descr:
        partes.append(f"dress: {dress_descr}")

    descricao_roupa = "; ".join(partes) if partes else "clothes"
    descricao_final = f"{gender_descr}, wearing {descricao_roupa}"

    detalhes = {
        "top_matches": top_matches[:top_k],
        "bottom_matches": bottom_matches[:top_k],
        "dress_matches": dress_matches[:top_k],
        "gender_matches": gender_matches[:top_k],
    }

    return descricao_final, detalhes  # TEXTO_FOCO + detalhes

# =====================================
# 4. Vetorização e query híbrida
# =====================================
def vetorizar_query(texto_ou_imagem, input_type="text"):
    """
    input_type='text' => texto_ou_imagem é string
    input_type='image' => texto_ou_imagem é caminho de imagem
    """
    if input_type == "text":
        inputs = processor(text=[texto_ou_imagem], return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            emb = model.get_text_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    elif input_type == "image":
        image = Image.open(texto_ou_imagem).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            emb = model.get_image_features(**inputs)
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
        return emb.cpu().numpy()[0]
    else:
        raise ValueError("input_type deve ser 'text' ou 'image'.")

def vetorizar_query_hibrida(query_texto, caminho_imagem):
    """
    Combina os vetores de texto e imagem numa única representação (vetor médio).
    """
    vetor_imagem = vetorizar_query(caminho_imagem, input_type='image')
    vetor_texto = vetorizar_query(query_texto, input_type='text')

    if vetor_imagem is not None and vetor_texto is not None:
        vetor_hibrido = (vetor_imagem + vetor_texto) / 2.0
        return vetor_hibrido
    elif vetor_imagem is not None:
        return vetor_imagem
    else:
        return None

# =====================================
# 5. Busca no catálogo (df_busca / VETORES_BUSCA)
# =====================================
def busca_hibrida_por_imagem(
    image_path,
    df_busca,
    vetores_busca,
    top_k=5,
    user_focus="outfit",   # "top", "bottom", "dress", "outfit"
    top_k_descr=3
):
    """
    O utilizador fornece a imagem e uma descrição textual do que quer (mapeada para user_focus).
    - Gera TEXTO_FOCO automaticamente com CLIP.
    - Cria vetor híbrido (texto + imagem).
    - Faz similaridade com VETORES_BUSCA e mostra top_k produtos.
    """
    if not os.path.exists(image_path):
        print("Imagem não encontrada:", image_path)
        return None

    image = Image.open(image_path).convert("RGB")

    # 1. Gerar TEXTO_FOCO com CLIP
    TEXTO_FOCO, detalhes = gerar_descricao_roupa(
        image=image,
        user_focus=user_focus,
        processor=processor,
        model=model,
        device=device,
        top_k=top_k_descr
    )

    print("\nTEXTO_FOCO gerado automaticamente:")
    print(TEXTO_FOCO)

    print("\nTop descritores usados (texto, prob):")
    print("Top:", detalhes["top_matches"])
    print("Bottom:", detalhes["bottom_matches"])
    print("Dress:", detalhes["dress_matches"])
    print("Gender:", detalhes["gender_matches"])

    # 2. Vetor híbrido
    vetor_hibrido = vetorizar_query_hibrida(TEXTO_FOCO, image_path)
    if vetor_hibrido is None:
        print("Falha ao gerar vetor híbrido.")
        return None

    vetor_busca_formatado = vetor_hibrido.reshape(1, -1)

    # 3. Similaridade com catálogo
    similaridades = cosine_similarity(vetor_busca_formatado, vetores_busca).flatten()
    indices_ranking = np.argsort(similaridades)[::-1]
    top_indices = indices_ranking[:top_k]

    resultados = df_busca.iloc[top_indices].copy()
    resultados['similaridade'] = similaridades[top_indices]

    print("\n--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---\n")
    for rank, (idx, produto) in enumerate(resultados.iterrows(), start=1):
        caminho_img = produto['caminho_imagem_local']
        print(f"[RANK {rank}] Produto: {produto['name']}")
        print(f"    Similaridade: {produto['similaridade']:.4f}")
        print(f"    Categoria/terms: {produto.get('terms', 'N/A')}")

        if os.path.exists(caminho_img):
            print(f"    A abrir imagem (Rank {rank}): {os.path.basename(caminho_img)}")
            try:
                Image.open(caminho_img).show()
            except Exception as e:
                print(f"    Erro ao abrir imagem: {e}")
        else:
            print("    Caminho de imagem não existe:", caminho_img)

        print("-" * 50)

    return resultados, TEXTO_FOCO, detalhes

# =====================================
# 6. Loop interativo (imagem + texto livre do utilizador)
# =====================================
if __name__ == "__main__":
    # df_busca e VETORES_BUSCA têm de estar definidos antes ou importados aqui.

    while True:
        image_path = input("\nIndica o caminho da imagem (ou 'sair'): ").strip()
        if image_path.lower() == "sair":
            break

        if not os.path.exists(image_path):
            print("Imagem não encontrada, tenta de novo.")
            continue

        user_text = input(
            "\nO que queres procurar? (ex: 'sweat', 't-shirt', 'pants', 'dress', 'outfit'):\n> "
        ).strip()

        user_focus = infer_user_focus(user_text)
        print(f"Interpretado como foco: {user_focus}")

        try:
            resultados, texto_foco, detalhes = busca_hibrida_por_imagem(
                image_path=image_path,
                df_busca=df_busca,
                vetores_busca=VETORES_BUSCA,
                top_k=5,
                user_focus=user_focus,
                top_k_descr=3
            )
        except Exception as e:
            print("Erro na busca híbrida:", e)


Interpretado como foco: dress

TEXTO_FOCO gerado automaticamente:
a woman (0.537), wearing dress: green dress, cotton, ankle length (0.525), grey dress, silk, ankle length (0.369), green dress, silk, ankle length (0.344)

Top descritores usados (texto, prob):
Top: []
Bottom: []
Dress: [('green dress, cotton, ankle length', 0.5250544548034668), ('grey dress, silk, ankle length', 0.36928650736808777), ('green dress, silk, ankle length', 0.3444611430168152)]
Gender: [('a woman', 0.5366086959838867), ('a girl', 0.45396989583969116), ('a man', 0.006011910270899534)]

--- RESULTADOS BUSCA HÍBRIDA (Imagem + Texto gerado) ---

[RANK 1] Produto: ASYMMETRICAL DRESS WITH RUCHING
    Similaridade: 0.6813
    Categoria/terms: dresses
    A abrir imagem (Rank 1): 98427b90-5fed-4205-af1b-65e88ba5c7df.jpg
--------------------------------------------------
[RANK 2] Produto: POLYAMIDE OFF-THE-SHOULDER DRESS
    Similaridade: 0.6605
    Categoria/terms: dresses
    A abrir imagem (Rank 2): 96fdf0a7-a62e-